In [ ]:
import pandas as pd
from pandas import DataFrame, Series
import numpy as np
import string

from clean_data import*
from add_summary import*
from find_category import*

In [2]:
candidate=DataFrame(columns=["id", "name","state_fk","date_birth", "place_birth", "position", "URL_photo", "party"])

candidate1=pd.DataFrame([[1, "Don Blankenship",1, "March 14, 1950", "Stopover, Kentucky", "Coal magnate", "https://i2.wp.com/woay.tv/wp-content/uploads/2017/05/GTY_don_blankenship_jt_160406_4x3_992.jpg?resize=735%2C400&ssl=1","Republican" ],
                     [2, "Bo Copley",1, "", " Dingess, West Virginia", "","https://heavyeditorial.files.wordpress.com/2017/05/bo-copley-e1494012597984.jpg?quality=65&strip=all&w=780&strip=all","Republican" ],
                     [3, "Evan Jenkins",1, "September 12, 1960", "Huntington, West Virginia","U.S. House of Representatives","https://www.congress.gov/img/member/114_rp_wv_3_jenkins_evan.jpg","Republican"  ],
                     [4, "Joe Manchin",1, " August 24, 1947", "Farmington, West Virginia", "United States Senator", "https://fi.wikipedia.org/wiki/Joe_Manchin#/media/File:Senator_Manchin.jpg", "Democrat"],
                     [5, "Patrick Morrisey",1,"December 21, 1967","Brooklyn, New York City, NY","West Virginia Attorney General since 2013","http://wvmetronews.com/wordpress/wp-content/uploads/2016/02/Patrick-Morrisey.jpg?x43308","Republican"  ],
                     [6, "Jack Newbrough",1, "", "","U.S. Navy veteran","http://s3.amazonaws.com/ogden_images/www.weirtondailytimes.com/images/2017/09/23190845/Newbrough-003-1100x825.jpg" ,"Republican" ],
                     [7, "Paula Jean Swearengin", 1, "born 1973/74","Mullens, West Virginia","","https://cbbba9d899c5f8ae6108-e379174d4d25dcb78174c6cab0a88cea.ssl.cf1.rackcdn.com/1275564-l.jpg", "Democrat"],
                     [8, "Tom Willis",1, "","","","https://api.ballotpedia.org/v3/thumbnail/200/300/crop/best/2C512069-7298-4BAC-B9F3-6BFECDC2FB57.jpeg","Republican" ],
                     [9, "Nicholas Freitas", 2, "", "", "","","Republican"],
                     [10, "E. W. Jackson", 2,"","","","","Republican"],
                     [11, "Tim Kaine", 2, "", "", "", "", "Democrat"],
                     [12, "Corey Stewart", 2, "", "", "", "", "Republican"],
                     [13, "Matt Waters", 2, "", "", "", "", "Libertarian"],
                     [14, "Carl Bible", 3, "", "", "", "", "Independent"],
                     [15, "Ted Cruz", 3, "", "", "", "", "Republican"],
                     [16, "Bob McNeil", 3, "", "", "", "", "American Citizen Party"],
                     [17, "Beto O'Rourke", 3, "", "", "", "", "Democrat"]],
                        columns=["id", "name","state_fk","date_birth", "place_birth", "position","URL_photo", "party"])
candidate=candidate.append(candidate1)


In [3]:
all_candidates = []
for index,row in candidate.iterrows():
    all_candidates.append(row['name'])

In [4]:
## script is hardcoded to take in data from the RAW_DATA file
# this function takes in a list of candidates (string form) and a name of the output json file
structure_data(all_candidates, "ALL_DATA.json")

In [5]:
# this script takes an input file and creates a new json
add_summary("ALL_DATA.json", "ALL_DATA_SUMM.json")

In [6]:
all_data = pd.read_json("ALL_DATA_SUMM.json")

In [ ]:
## category analysis

In [7]:
# call script to generate categories and 

all_candidates_str = [x.split() for x in all_candidates]
all_candidates_str = [x.lower() for y in all_candidates_str for x in y]
all_candidates_str.append("ORourke".lower())
extra_words = ["like", "it’s", "uh", "going", "that’s", "think", "actually", "kind", "…", "know", "come", "u", "really"
                       ,"mr", "june", "july", "august", "aug", "festival", "theater", "music", "concert", "dance", "performance"
                       , "jazz", "band", "arts", "art","museum","painting", "artist", "exhibition","gallery", "ms", "sculpture"
                       , "tell", "also","thats", "im", "album", "film", "movie", "opera", "cookbook", "orchestra", "play"
                       , "street", "design", "photograph", "drawing", "collection", "street", "design", "direct", "jan", "may","summer"
                       , "season", "production", "park", "oct", "show", "quartet", "series", "may", "star", "feature", "tonight"
                       , "avenue", "village", "production", "summer", "season", "premiere","ballet", "photograph", "drawing", "boat"
                       , "symphony", "shakespeare", "repertory","pop", "perform", "perform", "night", "classical", "guest", "musical"
                       ,"january", "jan", "february", "feb", "march", "mar", "april", "apr","may", "june", "july", "august", "aug"
                       ,"september", "sept", "october", "oct", "november", "nov", "december", "dec", "literature"]

extra_words.extend(all_candidates_str)


In [8]:
corpus_text = []
for index,row in all_data.iterrows():
    corpus_text.append(row['article_text'])

In [ ]:
corpus_titles = []
for index,row in all_data.iterrows():
    corpus_titles.append(row['articles_title'])

In [9]:
relevant_words_text = find_categories_and_top20(corpus_text, extra_words)

In [ ]:
relevant_words_title = find_categories_and_top20(corpus_titles, extra_words)

In [ ]:
create_lda_page(relevant_words_text, "lda_text.html")

In [ ]:
create_lda_page(relevant_words_title, "lda_title.html")

In [ ]:
#b = [[2*x for y in a] for x in y]
#c = [[2*x for x in y] for y in a]
#[[int(x) for x in feet] for x in feet]

In [ ]:
## tf-idf vectorization process

In [10]:
relevant_words_text_string = [" ".join(y) for y in relevant_words_text]

In [11]:
# create dictionary: map candidate to list of articles they appear in
articles_per_candidate = {}

for i in range(len(all_data)):
    candidate_name = all_data["first_name"][i] + " " + all_data["last_name"][i]
    if candidate_name in articles_per_candidate:
        articles_per_candidate[candidate_name].append(i)
    else:
        articles_per_candidate[candidate_name] = [i]
    

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(analyzer='word', ngram_range=(1,1), min_df = 0, stop_words = 'english')

tfidf_matrix =  tf.fit_transform(relevant_words_text_string)
feature_names = tf.get_feature_names()

dense = tfidf_matrix.todense()


In [13]:
top_20_per_candidate = {}

for person in articles_per_candidate:
    candidate_articles = articles_per_candidate[person]
    summed_values = [0]*len(feature_names)
    
    phrase_scores = []
    sorted_scores = []
    word_val_pair = []
    for article_id in candidate_articles:
        article_vector = dense[article_id].tolist()[0]
        summed_values = [x+y for x,y in zip(summed_values,article_vector)]
        
    pairs = list(zip(range(0, len(feature_names)), summed_values))

    phrase_scores = [pair for pair in pairs if pair[1] > 0]
        
    sorted_scores = sorted(phrase_scores, key=lambda x: x[1], reverse=True)[0:20]

    word_val_pair = []
    for phrase, score in [(feature_names[word_id], score) for (word_id, score) in sorted_scores][:]:
        word_val_pair.append((phrase,score))
    
    top_20_per_candidate[person] = word_val_pair

Bo Copley


In [14]:
top_20_per_candidate

{"Beto O'Rourke": [('rep', 4.175245438168791),
  ('democrat', 2.852917810250115),
  ('texas', 2.5415666310047373),
  ('republican', 2.5241566464505234),
  ('state', 2.2917785345340187),
  ('trump', 2.120021172763093),
  ('say', 2.0560374719281893),
  ('primary', 1.9749754456594555),
  ('race', 1.921741218333795),
  ('democratic', 1.8332502610103554),
  ('win', 1.7176413602903562),
  ('sen', 1.6755793350425225),
  ('seat', 1.521439184481938),
  ('incumbent', 1.4617482530834573),
  ('veteran', 1.386488659950414),
  ('election', 1.3537148831083983),
  ('run', 1.3433226753107186),
  ('campaign', 1.261145940539877),
  ('senate', 1.2298122135531726),
  ('vote', 1.1471382030235528)],
 'Bo Copley': [('coal', 3.0903225060595556),
  ('pm', 1.736928202829196),
  ('mrs', 1.3563290888562776),
  ('west', 1.3133860654298262),
  ('clinton', 1.2894071186200653),
  ('church', 1.0504855044151347),
  ('say', 0.9999949350277741),
  ('miner', 0.9253563282217441),
  ('trump', 0.9163083270742752),
  ('st', 0.

In [ ]:
from sklearn.cluster import KMeans

k = 10
km = KMeans(n_clusters=k, init='k-means++', max_iter=100, n_init=5,
                verbose=1)
km.fit(tfidf_matrix)

In [ ]:
from yellowbrick.text import TSNEVisualizer

tsne = TSNEVisualizer()
tsne.fit(tfidf_matrix)
tsne.poof()